In [ ]:
import numpy as np
import pandas as pd, re, requests
from itertools import combinations
from scipy.sparse import dok_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [3]:
dataset_dir = ''
df = pd.read_csv("diagnoses_icd10.csv", index_col=0)
admissions = pd.read_csv(os.path.join(dataset_dir, "admissions.csv"))
df = pd.merge(
    df,
    admissions[['hadm_id', 'admittime', 'dischtime', 'deathtime']],
    on='hadm_id'
)

In [4]:
n9  = (df.icd_version == 9).sum()
nun = df["icd10_code"].isna() & (df.icd_version == 9)

In [5]:
mapped = df["icd10_code"].notnull() & (df["icd_version"] == 9)
unmapped = df["icd10_code"].isna() & (df["icd_version"] == 9)

In [6]:
def is_onco(code: str) -> int:
    """
    Malignant neoplasms  C00–C97
    In-situ + uncertain   D00–D09, D37–D48
    Берём три ведущих символа и проверяем диапазоны.
    """
    c = code.upper()
    if c[0] == "C":
        return 1
    if re.match(r"D0[0-9]|D[3-4][7-8]", c):
        return 1
    return 0

df["is_oncology"] = df["icd10_code"].map(is_onco).astype("int8")
df["category"] = df["icd10_code"].str[:3]
df.head(5)

,subject_id,hadm_id,seq_num,icd_code,icd_version,icd10_code,icd10_category,admittime,dischtime,deathtime,is_oncology,category
0,10000032,22595853,1,5723,9,K766,K76,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,0,K76
1,10000032,22595853,2,78959,9,R188,R18,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,0,R18
2,10000032,22595853,3,5715,9,K740,K74,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,0,K74
3,10000032,22595853,4,07070,9,B1920,B19,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,0,B19
4,10000032,22595853,5,496,9,J449,J44,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,0,J44


In [13]:
df = df.sort_values(["subject_id", "hadm_id", "seq_num"])
df_clean = df.drop_duplicates(subset=["subject_id", "hadm_id", "category"], keep="first").copy()

In [ ]:
df_clean.to_csv("prepared_data.csv")